# Plugins

> Optional plugin integration for extensible settings systems

In [ ]:
#| default_exp plugins

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import tempfile
from pathlib import Path
from typing import Dict, Any, Optional, Protocol, runtime_checkable

# Import from cjm-fasthtml-plugins
from cjm_fasthtml_plugins.core.metadata import PluginMetadata

## Plugin Integration

This module provides integration with the `cjm-fasthtml-plugins` library for extensible settings systems.

- Uses `PluginMetadata` from `cjm-fasthtml-plugins`
- Defines `PluginRegistryProtocol` for registry compatibility
- Applications should use `UnifiedPluginRegistry` from `cjm-fasthtml-plugins` instead of `SimplePluginRegistry`

In [ ]:
# Cell removed - using PluginMetadata from cjm-fasthtml-plugins

In [ ]:
# Cell removed - PluginMetadata now imported from cjm-fasthtml-plugins
# The imported version includes additional features:
# - Execution mode tracking (local, cloud, subprocess, etc.)
# - Resource management (child PIDs, containers, cloud instances)
# - Lifecycle awareness support

## Plugin Registry Protocol

In [ ]:
#| export
@runtime_checkable
class PluginRegistryProtocol(Protocol):
    """Protocol that plugin registries should implement.
    
    This allows the settings library to work with any plugin system
    that implements these methods.
    """
    
    def get_plugin(self, unique_id: str) -> Optional[PluginMetadata]:
        """Get plugin metadata by unique ID."""
        ...
    
    def get_plugins_by_category(self, category: str) -> list[PluginMetadata]:
        """Get all plugins in a category."""
        ...
    
    def get_categories_with_plugins(self) -> list[str]:
        """Get all categories that have registered plugins."""
        ...
    
    def load_plugin_config(self, unique_id: str) -> Dict[str, Any]:
        """Load saved configuration for a plugin."""
        ...
    
    def save_plugin_config(self, unique_id: str, config: Dict[str, Any]) -> bool:
        """Save configuration for a plugin."""
        ...

In [ ]:
# Example: Using UnifiedPluginRegistry from cjm-fasthtml-plugins
from cjm_fasthtml_plugins.core.registry import UnifiedPluginRegistry

# Mock plugin manager for demonstration
from dataclasses import dataclass as dc

@dc
class MockPluginData:
    name: str
    version: str

class MockPluginManager:
    def discover_plugins(self):
        return [
            MockPluginData("plugin_a", "1.0.0"),
            MockPluginData("plugin_b", "1.0.0")
        ]
    def get_plugin_config_schema(self, name: str):
        return {
            "type": "object",
            "title": f"{name.title()} Configuration",
            "properties": {
                "enabled": {"type": "boolean", "default": True}
            }
        }

with tempfile.TemporaryDirectory() as tmpdir:
    # Create unified registry
    registry = UnifiedPluginRegistry(config_dir=Path(tmpdir))
    
    # Register a plugin manager for a category
    registry.register_plugin_manager(
        category="data_processing",
        manager=MockPluginManager(),
        display_name="Data Processing"
    )
    
    # The registry implements PluginRegistryProtocol
    print(f"Implements protocol: {isinstance(registry, PluginRegistryProtocol)}")
    print(f"Categories: {registry.get_categories_with_plugins()}")
    print(f"Plugins in category: {[p.name for p in registry.get_plugins_by_category('data_processing')]}")
    
    # Get a specific plugin
    plugin = registry.get_plugin("data_processing_plugin_a")
    print(f"\nPlugin metadata:")
    print(f"  Name: {plugin.name}")
    print(f"  Category: {plugin.category}")
    print(f"  Title: {plugin.title}")
    print(f"  Unique ID: {plugin.get_unique_id()}")
    
    # Save and load config
    registry.save_plugin_config("data_processing_plugin_a", {"enabled": False})
    loaded = registry.load_plugin_config("data_processing_plugin_a")
    print(f"\nSaved and loaded config: {loaded}")

Implements protocol: True
Categories: ['data_processing']
Plugins in category: ['plugin_a', 'plugin_b']

Plugin metadata:
  Name: plugin_a
  Category: data_processing
  Title: Plugin_A Configuration
  Unique ID: data_processing_plugin_a

Saved and loaded config: {'enabled': False}


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()